In [15]:
from fastai.vision.all import *
from pathlib import Path
import kaggle
import fastkaggle
from fastcore.parallel import *
import gc

In [2]:
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [3]:
creds='{"username":"atikarahmanpaddo","key":"1c7600647915e2491d5170e9995d948e"}'

In [4]:
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [5]:
if iskaggle:
    path = Path('../input/paddy-disease-classification')
    ! pip install -q datasets

In [8]:
path = Path('paddy-disease-classification')

In [9]:
tst_files = get_image_files(path/'test_images').sorted()

In [10]:
df = pd.read_csv(path/'train.csv')
df.label.value_counts()

normal                      1764
blast                       1738
hispa                       1594
dead_heart                  1442
tungro                      1088
brown_spot                   965
downy_mildew                 620
bacterial_leaf_blight        479
bacterial_leaf_streak        380
bacterial_panicle_blight     337
Name: label, dtype: int64

In [12]:
trn_path = path/'train_images'/'bacterial_panicle_blight'

In [13]:
def train(arch, size, item=Resize(480, method='squish'), accum=1, finetune=True, epochs=12):
    dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, item_tfms=item,
        batch_tfms=aug_transforms(size=size, min_scale=0.75), bs=64//accum)
    cbs = GradientAccumulation(64) if accum else []
    learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
    if finetune:
        learn.fine_tune(epochs, 0.01)
        return learn.tta(dl=dls.test_dl(tst_files))
    else:
        learn.unfreeze()
        learn.fit_one_cycle(epochs, 0.01)

In [14]:
#Using this model
train('convnext_small_in22k', 128, epochs=1, accum=1, finetune=False)

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_small_22k_224.pth" to /home/jupyter-apaddo/.cache/torch/hub/checkpoints/convnext_small_22k_224.pth


epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:02


In [16]:
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [19]:
report_gpu()

GPU:0
no processes are running


In [18]:
pip install pynvml

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 46 kB 5.0 MB/s             
Note: you may need to restart the kernel to use updated packages.


In [20]:
train('convnext_small_in22k', 128, epochs=1, accum=2, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:01


GPU:0
no processes are running


In [21]:
train('convnext_small_in22k', 128, epochs=1, accum=4, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:01


GPU:0
no processes are running


In [22]:
train('convnext_large_in22k', 224, epochs=1, accum=2, finetune=False)
report_gpu()

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_224.pth" to /home/jupyter-apaddo/.cache/torch/hub/checkpoints/convnext_large_22k_224.pth


epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:02


GPU:0
no processes are running


In [23]:
train('convnext_large_in22k', (320,240), epochs=1, accum=2, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:03


GPU:0
no processes are running


In [24]:
train('vit_large_patch16_224', 224, epochs=1, accum=2, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:03


GPU:0
no processes are running


In [25]:
train('swinv2_large_window12_192_22k', 192, epochs=1, accum=2, finetune=False)
report_gpu()

/home/jupyter-apaddo/.local/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v2.0.0/swinv2_large_patch4_window12_192_22k.pth" to /home/jupyter-apaddo/.cache/torch/hub/checkpoints/swinv2_large_patch4_window12_192_22k.pth


epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:03


GPU:0
no processes are running


In [26]:
train('swin_large_patch4_window7_224', 224, epochs=1, accum=2, finetune=False)
report_gpu()

Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_large_patch4_window7_224_22kto1k.pth" to /home/jupyter-apaddo/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth


epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:03


GPU:0
no processes are running


# Running the models

In [27]:
res = 640,480

In [28]:
res = 640,480
models = {
    'convnext_large_in22k': {
        (Resize(res), (320,224)),
    }, 'vit_large_patch16_224': {
        (Resize(480, method='squish'), 224),
        (Resize(res), 224),
    }, 'swinv2_large_window12_192_22k': {
        (Resize(480, method='squish'), 192),
        (Resize(res), 192),
    }, 'swin_large_patch4_window7_224': {
        (Resize(res), 224),
    }
}

In [29]:
trn_path = path/'train_images'

In [30]:
tta_res = []

for arch,details in models.items():
    for item,size in details:
        print('---',arch)
        print(size)
        print(item.name)
        tta_res.append(train(arch, size, item=item, accum=2)) #, epochs=1))
        gc.collect()
        torch.cuda.empty_cache()


--- convnext_large_in22k
(320, 224)
Resize -- {'size': (480, 640), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


epoch,train_loss,valid_loss,error_rate,time
0,0.851020,0.445202,0.143681,01:07


epoch,train_loss,valid_loss,error_rate,time
0,0.405347,0.250489,0.084575,01:32
1,0.314447,0.248336,0.071600,01:37
2,0.343457,0.283053,0.079769,01:39
3,0.242484,0.198108,0.063431,01:42
4,0.174880,0.165971,0.040365,01:44
5,0.109640,0.152491,0.037963,01:43
6,0.097029,0.140050,0.032677,01:50
7,0.056625,0.106634,0.026910,01:48
8,0.065862,0.102163,0.024507,01:47
9,0.032309,0.089721,0.023546,01:46


--- vit_large_patch16_224
224
Resize -- {'size': (480, 640), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


epoch,train_loss,valid_loss,error_rate,time
0,0.994280,0.705933,0.224892,01:29


...

--- swinv2_large_window12_192_22k
192
Resize -- {'size': (480, 480), 'method': 'squish', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


epoch,train_loss,valid_loss,error_rate,time
0,0.848845,0.659871,0.203748,01:34


epoch,train_loss,valid_loss,error_rate,time
0,0.428486,0.244431,0.073522,01:45
1,0.338651,0.258514,0.083614,01:45
2,0.356694,0.240969,0.072081,01:49
3,0.297636,0.278098,0.088900,01:46
4,0.223703,0.179908,0.051898,01:44
5,0.159512,0.163317,0.044690,01:45
6,0.134826,0.122825,0.034599,01:46
7,0.095183,0.121355,0.034118,01:46
8,0.062180,0.106432,0.024507,01:44


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

In [31]:
save_pickle('tta_res.pkl', tta_res)

In [33]:
tta_prs = first(zip(*tta_res))

In [34]:
tta_prs += tta_prs[1:3]

In [35]:
avg_pr = torch.stack(tta_prs).mean(0)
avg_pr.shape

torch.Size([3469, 10])